Trigger type is default -which is contionusly running microbatches as one batch finishes
This Notebok is reading data from delta as stream.when ever there is any change in delta table in the form of append this stream application captures that increamental data and writes it to sink
Drawback of this use case is -it cannot handle any changes in the upstream source table that is not an append and throws an exception

Let us see two  scenarios -
1.when we start an applcation it reads the intial snapshot of data.In this case we had manually inserted 10 records from orders data -refer upstream_data writer notebook .in first microbatch 10 recods are processed from the inital data and in the following microbatch the new 100 records are  processed.
 
streaming_query.recentProgress will help in getting the status of all the micro batches  - for example -details of first micro batch.

We can check the number of rows processed using numInputRows key in this dictionary

{'id': 'fb3a7f06-e606-416d-964b-7b257c03db92',
  'runId': '238b82b4-2b3d-4468-8618-6984e01e50ae',
  'name': None,
  'timestamp': '2024-10-19T12:26:32.123Z',
  'batchId': 0,
  'numInputRows': 10,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 1.5276504735716467,
  'durationMs': {'addBatch': 4234,
   'commitOffsets': 118,
   'getBatch': 54,
   'latestOffset': 1749,
   'queryPlanning': 43,
   'triggerExecution': 6546,
   'walCommit': 181},
  'stateOperators': [],
  'sources': [{'description': 'DeltaSource[dbfs:/FileStore/tables/delta/orders]',
    'startOffset': None,
    'endOffset': {'sourceVersion': 1,
     'reservoirId': '4af1a0f9-b71a-4acc-b926-00fefbed9ce1',
     'reservoirVersion': 0,
     'index': 0,
     'isStartingVersion': True},
    'latestOffset': None,
    'numInputRows': 10,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 1.5276504735716467,
    'metrics': {'numBytesOutstanding': '0', 'numFilesOutstanding': '0'}}],
  'sink': {'description': 'DeltaSink[/FileStore/tables/delta/stream_orders]',
   'numOutputRows': -1}}



similarly when the source table gets the new data ,straem query reads it as a microbatch and writes it to sink.
{'id': 'fb3a7f06-e606-416d-964b-7b257c03db92',
  'runId': '238b82b4-2b3d-4468-8618-6984e01e50ae',
  'name': None,
  'timestamp': '2024-10-19T12:32:24.931Z',
  'batchId': 1,
  'numInputRows': 100,
  'inputRowsPerSecond': 181.15942028985506,
  'processedRowsPerSecond': 16.570008285004143,
  'durationMs': {'addBatch': 5283,
   'commitOffsets': 121,
   'getBatch': 192,
   'latestOffset': 152,
   'queryPlanning': 19,
   'triggerExecution': 6035,
   'walCommit': 261},
  'stateOperators': [],
  'sources': [{'description': 'DeltaSource[dbfs:/FileStore/tables/delta/orders]',
    'startOffset': {'sourceVersion': 1,
     'reservoirId': '4af1a0f9-b71a-4acc-b926-00fefbed9ce1',
     'reservoirVersion': 0,
     'index': 0,
     'isStartingVersion': True},
    'endOffset': {'sourceVersion': 1,
     'reservoirId': '4af1a0f9-b71a-4acc-b926-00fefbed9ce1',
     'reservoirVersion': 2,
     'index': -1,
     'isStartingVersion': False},
    'latestOffset': None,
    'numInputRows': 100,
    'inputRowsPerSecond': 181.15942028985506,
    'processedRowsPerSecond': 16.570008285004143,
    'metrics': {'numBytesOutstanding': '0', 'numFilesOutstanding': '0'}}],
  'sink': {'description': 'DeltaSink[/FileStore/tables/delta/stream_orders]',
   'numOutputRows': -1}}


Scenario 2 -when there is an update to the source table the below query fails and it is not propogated to down stream. we will get the below error

com.databricks.sql.transaction.tahoe.DeltaUnsupportedOperationException: Detected a data update (for example part-00000-f4791fdb-58a9-495c-b2e9-a229c8c0eb05-c000.snappy.parquet) in the source table at version 3. This is currently not supported. If you'd like to ignore updates, set the option 'skipChangeCommits' to 'true'. If you would like the data update to be reflected, please restart this query with a fresh checkpoint directory. The source table can be found at path dbfs:/FileStore/tables/delta/orders.






In [0]:
from pyspark.sql.functions import *

In [0]:
orders_stream=spark.readStream\
    .format("delta")\
    .option("path","/FileStore/tables/delta/orders")\
    .load()

orders_stream1=orders_stream.withColumn("Processing_timestamp",current_timestamp())


In [0]:
path="/FileStore/tables/delta/stream_orders"
streaming_query=orders_stream1.writeStream\
    .format("delta")\
    .option("path",path)\
    .option("checkpointLocation",f"{path}/_checkpoint")\
    .start()